![](christmas_tree.png)

In [ ]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import numpy as np

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
data = todays_puzzle.input_data
todays_examples = todays_puzzle.examples

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_a="12")

# Part A

In [ ]:
def multiply_robots_in_quadrants(robot_positions, grid_size: tuple):
    grid = np.zeros(grid_size)
    for robot in robot_positions:
        grid[robot[0], robot[1]] += 1
    q_shape = (grid_size[0] // 2, grid_size[1] // 2)
    q1 = np.sum(grid[: q_shape[0], : q_shape[1]])
    q2 = np.sum(grid[: q_shape[0], q_shape[1] + 1 :])
    q3 = np.sum(grid[q_shape[0] + 1 :, : q_shape[1]])
    q4 = np.sum(grid[q_shape[0] + 1 :, q_shape[1] + 1 :])

    return int(q1 * q2 * q3 * q4)

In [ ]:
def part_a(data: str, grid_size: tuple) -> str:
    N = 100
    robot_positions = re.findall(r"p=(\d+),(\d+)", data)
    robot_velocities = re.findall(r"v=(-?\d+),(-?\d+)", data)
    robot_positions = np.array(robot_positions, dtype=int)
    robot_velocities = np.array(robot_velocities, dtype=int)
    for i in range(len(robot_positions)):
        robot_positions[i] += robot_velocities[i] * N
        robot_positions[i] = robot_positions[i] % grid_size
    result = multiply_robots_in_quadrants(robot_positions, grid_size)
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data,(11,7))} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data), (11, 7)) == example.answer_a
submit(part_a(data, (101, 103)), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
import plotly.express as px

In [ ]:
def iterate_robots(robot_positions, robot_velocities, grid_size: tuple, n=1):
    robot_positions += robot_velocities * n
    for i in range(len(robot_positions)):
        robot_positions[i] = robot_positions[i] % grid_size


def robot_positions_to_grid(robot_positions, grid_size: tuple):
    grid = np.zeros(grid_size)
    for x, y in robot_positions:
        grid[(x, y)] += 1
    return grid


def visualise_robots(robot_positions, grid_size: tuple):
    grid = robot_positions_to_grid(robot_positions, grid_size)
    fig = px.imshow(grid.T)
    return fig

In [ ]:
def get_robots_positions_velocity_from_data(data: str):
    robot_positions = re.findall(r"p=(\d+),(\d+)", data)
    robot_velocities = re.findall(r"v=(-?\d+),(-?\d+)", data)
    robot_positions = np.array(robot_positions, dtype=int)
    robot_velocities = np.array(robot_velocities, dtype=int)
    return robot_positions, robot_velocities

## Check the functions

In [ ]:
robot_positions, robot_velocities = get_robots_positions_velocity_from_data(
    todays_examples[0].input_data
)

In [ ]:
visualise_robots(robot_positions, (11, 7))

In [ ]:
iterate_robots(robot_positions, robot_velocities, (11, 7), 100)
visualise_robots(robot_positions, (11, 7))

## Check the period of the robots configuration

In [ ]:
robot_positions, robot_velocities = get_robots_positions_velocity_from_data(data)
robot_initial_position = robot_positions.copy()

for iteration in range(1, 20001):
    iterate_robots(robot_positions, robot_velocities, (101, 103))
    if (robot_initial_position == robot_positions).all():
        print(f"{iteration=}")
        break

## Check the cases when robots are in unique positions

In [ ]:
robot_positions, robot_velocities = get_robots_positions_velocity_from_data(data)

for iteration in range(1, 20001):
    iterate_robots(robot_positions, robot_velocities, (101, 103))
    grid = robot_positions_to_grid(robot_positions, (101, 103))
    if np.max(grid) == 1:
        print(f"solution, at ", iteration)
    if (robot_initial_position == robot_positions).all():
        print("repeated input at ", iteration)
        break

## Verify the solution

In [ ]:
robot_positions, robot_velocities = get_robots_positions_velocity_from_data(data)

iterate_robots(robot_positions, robot_velocities, (101, 103), n=7753)
visualise_robots(robot_positions, (101, 103))